In [1]:
import glob
import sys
import numpy as np

from enterprise.pulsar import Pulsar
from enterprise.signals import signal_base
from enterprise.signals import parameter
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals

import dpdm_delay

# import customized files to introduce 2T modes.
import gp_bases_new as gp_bases
import pl_prior as gp_priors

from enterprise.signals import selections

import multiprocess
from PTMCMCSampler.PTMCMCSampler import PTSampler


You'll need sksparse for get_coefficients() with common signals!
Cannot import PINT? Meh...
Do not have mpi4py package.


In [2]:
datadir = 'DR2'

parfiles = sorted(glob.glob(datadir + '/newpars/*.par'))
timfiles = sorted(glob.glob(datadir + '/toas/*.tim'))

In [3]:
psrs=[]
for ipsr in range(len(parfiles)):
    psr = Pulsar(parfiles[ipsr], timfiles[ipsr])
    psrs.append(psr)
    print(psr.name)

J0437-4715
J0613-0200
J0711-6830


J1017-7156
J1022+1001
J1024-0719
J1045-4509


J1125-6014


J1446-4701


J1545-4550
J1600-3053
J1603-7202
J1643-1224
J1713+0747
J1730-2304
J1732-5049
J1744-1134


J1824-2452A
J1832-0836
J1857+0943
J1909-3744
J1939+2134
J2124-3358
J2129-5721
J2145-0750


J2241-5236


In [5]:
# red noise
nmodes = 30


log10_A = parameter.Constant()
gamma = parameter.Constant()
pl = gp_priors.powerlaw(log10_A=log10_A, gamma=gamma)
dm_basis = gp_bases.createfourierdesignmatrix_dm(nmodes=nmodes)
red_basis = gp_bases.createfourierdesignmatrix_red(nmodes=nmodes)
selection = selections.Selection(selections.by_band)

dmn = gp_signals.BasisGP(pl, dm_basis, name='dm', coefficients=False)
spn = gp_signals.BasisGP(pl, red_basis, name='sp',coefficients=False)
bdn = gp_signals.BasisGP(pl, red_basis, name='bd',coefficients=False,selection=selection)

# white noise
backend = selections.Selection(selections.by_backend)
efac = parameter.Constant()
equad = parameter.Constant()

ef = white_signals.MeasurementNoise(efac=efac,selection=backend)
eq = white_signals.EquadNoise(log10_equad=equad,selection=backend)
wnb = ef + eq

# timing model
tm = gp_signals.TimingModel()

# Bayes ephmeris
eph = deterministic_signals.PhysicalEphemerisSignal(use_epoch_toas=True)


# include best fit parameters
Dict = np.load('noise_params/noisepars_m'+str(nmodes)+'.npy',allow_pickle=True,encoding='bytes').item()

## Prepare for Bayes Limits

In [11]:
def single_task(log10_ma, name, ret, model, test=False):
    
        np.random.seed()
    
        log10_eps = parameter.Uniform(-28.0, -16.0)('x_dp_log10_eps')
        dp = dpdm_delay.dpdm_block(model=model,log10_ma=log10_ma,log10_eps=log10_eps)

        model0  = tm + wnb + dmn + spn + eph + dp
        model1  = tm + wnb + dmn + spn + eph + dp #+ bdn

        signal=[]
        for psr in psrs:
            if psr.name in ['J0437-4715','J1939+2134']:
                signal.append( model1( psr ) )
            else:
                signal.append( model0( psr ) )

        pta = signal_base.PTA(signal)
        pta.set_default_params(Dict)
        if test == True:
            return pta

        #outDir="bayes/"+model+"_m"+str(nmodes)+"/n"+str(i).zfill(3)+'/'
        outDir="bayes/"+model+"_m"+str(nmodes)+'/'+name+'/'
#============================
        # to make the initial values
        xs = {par.name:par.sample() for par in pta.params}
        for parname in Dict.keys():
            if parname in xs.keys():
                xs[parname] = Dict[parname]
        
        x0 = np.hstack([xs[key] for key in sorted(xs.keys())])    
        ndim = len(x0)
#============================

        x0[-19:-8] = 0.
        x0[-8] = -26.
        
        groups=[range(ndim),range(ndim-8,ndim)]
        
        print(pta.get_lnlikelihood(x0))
        
        
        covdiag = np.ones(ndim)*0.01

        sampler = PTSampler(ndim, pta.get_lnlikelihood, pta.get_lnprior,
                        cov=np.diag(covdiag),outDir=outDir,groups=groups,resume=False)

        sampler.sample(x0, 40000, isave=1000)
        
        #chain=np.loadtxt(outDir+'/chain_1.txt')

        #logep=chain[int(0.8*len(chain)):,-12]

        #ret.value = np.percentile(logep,95)


## Run the code

In [14]:
a = 0
b = 20

print(a,b,nmodes)

jobs = []
RETs={}

for i in range(a,b): 

    RETs[i] = multiprocess.Manager().Value('i',0)

    logmamin = -23.5 + 2.5 * float(i) / float(125)
    logmamax = -23.5 + 2.5 * float(i+1) / float(125)
    
    log10_ma = parameter.Uniform(logmamin,logmamax)('x_dp_log10_ma')#n=0~99

    p = multiprocess.Process(target=single_task, args=(log10_ma, 'C'+str(i).zfill(3) , RETs[i] , "U1B" ))

    jobs.append(p)

    p.start()


for p in jobs:

    p.join()
    


0 20 30
model:U1B
model:U1B
model:U1B
model:U1B
model:U1B
model:U1B
model:U1B
model:U1B
model:U1B
model:U1B
model:U1B
model:U1B
model:U1B
model:U1B
model:U1B
model:U1B
model:U1B
model:U1B
model:U1B
model:U1B


INFO: enterprise.signals.signal_base: Setting J0437-4715_CASPSR_40CM_efac to 1.381236242017895
INFO: enterprise.signals.signal_base: Setting J0437-4715_CPSR2_10CM_efac to 1.115906838115193
INFO: enterprise.signals.signal_base: Setting J0437-4715_CPSR2_20CM_efac to 1.0345211790023054
INFO: enterprise.signals.signal_base: Setting J0437-4715_CPSR2_50CM_efac to 1.1062530554266004
INFO: enterprise.signals.signal_base: Setting J0437-4715_PDFB1_10CM_efac to 1.016602295219038
INFO: enterprise.signals.signal_base: Setting J0437-4715_PDFB1_1433_efac to 1.9195000536643811
INFO: enterprise.signals.signal_base: Setting J0437-4715_PDFB1_20CM_efac to 1.7183091165231414
INFO: enterprise.signals.signal_base: Setting J0437-4715_PDFB1_early_10CM_efac to 0.4323216683443894
INFO: enterprise.signals.signal_base: Setting J0437-4715_PDFB1_early_20CM_efac to 0.3127772717193401
INFO: enterprise.signals.signal_base: Setting J0437-4715_PDFB_10CM_efac to 0.04265640043117522
INFO: enterprise.signals.signal_base: Se

INFO: enterprise.signals.signal_base: Setting J0711-6830_sp_log10_A to -12.93563199011663
INFO: enterprise.signals.signal_base: Setting J0711-6830_sp_gamma to 1.2765949897692337
INFO: enterprise.signals.signal_base: Setting J1017-7156_CASPSR_40CM_efac to 0.8650848006410282
INFO: enterprise.signals.signal_base: Setting J1017-7156_PDFB_10CM_efac to 0.9393763347908762
INFO: enterprise.signals.signal_base: Setting J1017-7156_PDFB_20CM_efac to 0.9853721501815326
INFO: enterprise.signals.signal_base: Setting J1017-7156_PDFB_40CM_efac to 1.0224397199778663
INFO: enterprise.signals.signal_base: Setting J1017-7156_CASPSR_40CM_log10_equad to -5.951413841970355
INFO: enterprise.signals.signal_base: Setting J1017-7156_PDFB_10CM_log10_equad to -5.7853971211858
INFO: enterprise.signals.signal_base: Setting J1017-7156_PDFB_20CM_log10_equad to -6.583028053922432
INFO: enterprise.signals.signal_base: Setting J1017-7156_PDFB_40CM_log10_equad to -9.82983560260149
INFO: enterprise.signals.signal_base: Set

INFO: enterprise.signals.signal_base: Setting J1045-4509_PDFB1_10CM_log10_equad to -8.748324717018969
INFO: enterprise.signals.signal_base: Setting J1045-4509_PDFB1_1433_log10_equad to -6.676546444436767
INFO: enterprise.signals.signal_base: Setting J1045-4509_PDFB1_20CM_log10_equad to -9.171908079755754
INFO: enterprise.signals.signal_base: Setting J1045-4509_PDFB1_early_10CM_log10_equad to -8.773222214303052
INFO: enterprise.signals.signal_base: Setting J1045-4509_PDFB1_early_20CM_log10_equad to -7.0837271150118
INFO: enterprise.signals.signal_base: Setting J1045-4509_PDFB_10CM_log10_equad to -6.482551534715826
INFO: enterprise.signals.signal_base: Setting J1045-4509_PDFB_20CM_log10_equad to -6.073325284295592
INFO: enterprise.signals.signal_base: Setting J1045-4509_PDFB_40CM_log10_equad to -5.672639602135008
INFO: enterprise.signals.signal_base: Setting J1045-4509_WBCORR_10CM_log10_equad to -5.429308017357667
INFO: enterprise.signals.signal_base: Setting J1045-4509_dm_log10_A to -12

INFO: enterprise.signals.signal_base: Setting J1603-7202_PDFB1_early_10CM_efac to 0.9116885979685375
INFO: enterprise.signals.signal_base: Setting J1603-7202_PDFB1_early_20CM_efac to 0.9802833103313698
INFO: enterprise.signals.signal_base: Setting J1603-7202_PDFB_10CM_efac to 0.9750524695080585
INFO: enterprise.signals.signal_base: Setting J1603-7202_PDFB_20CM_efac to 0.9482944522918536
INFO: enterprise.signals.signal_base: Setting J1603-7202_PDFB_40CM_efac to 1.0172695890388483
INFO: enterprise.signals.signal_base: Setting J1603-7202_WBCORR_10CM_efac to 1.010155146825575
INFO: enterprise.signals.signal_base: Setting J1603-7202_CASPSR_40CM_log10_equad to -7.082837836610442
INFO: enterprise.signals.signal_base: Setting J1603-7202_CPSR2_20CM_log10_equad to -6.5774569689362785
INFO: enterprise.signals.signal_base: Setting J1603-7202_CPSR2_50CM_log10_equad to -7.894012205431749
INFO: enterprise.signals.signal_base: Setting J1603-7202_PDFB1_10CM_log10_equad to -8.81972026871363
INFO: enterp

...

1870824.9195492782


INFO: enterprise.signals.signal_base: Setting J2241-5236_sp_log10_A to -13.375127732146677
INFO: enterprise.signals.signal_base: Setting J2241-5236_sp_gamma to 0.19619627264930747
/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/parameter.py:64: RuntimeWarning: divide by zero encountered in log
  logpdf = np.log(self.prior(value, **kwargs))
/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/utils.py:796: RuntimeWarning: divide by zero encountered in true_divide
  nmat = Mmat / norm
/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/utils.py:796: RuntimeWarning: invalid value encountered in true_divide
  nmat = Mmat / norm


1870825.0701644286


/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/parameter.py:64: RuntimeWarning: divide by zero encountered in log
  logpdf = np.log(self.prior(value, **kwargs))


Finished 25.00 percent in 1373.020823 s Acceptance rate = 0.374267Adding DE jump with weight 20
Finished 25.00 percent in 1379.505193 s Acceptance rate = 0.3788Adding DE jump with weight 20
Finished 22.50 percent in 1381.916266 s Acceptance rate = 0.386222Adding DE jump with weight 20
Finished 25.00 percent in 1385.879433 s Acceptance rate = 0.3902Adding DE jump with weight 20
Finished 25.00 percent in 1398.565562 s Acceptance rate = 0.3837Adding DE jump with weight 20
Finished 25.00 percent in 1399.704007 s Acceptance rate = 0.3812Adding DE jump with weight 20
Finished 25.00 percent in 1401.696077 s Acceptance rate = 0.3678Adding DE jump with weight 20
Adding DE jump with weight 20
Finished 25.00 percent in 1405.875404 s Acceptance rate = 0.3865Adding DE jump with weight 20
Finished 25.00 percent in 1411.104945 s Acceptance rate = 0.3801Adding DE jump with weight 20
Finished 25.00 percent in 1414.604167 s Acceptance rate = 0.3954Adding DE jump with weight 20
Finished 25.00 percent in 

# Stop Here.

## Check EPH effect

In [ ]:
dp = dpdm_delay.dpdm_block(model='U1B')

model0  = tm + wnb + dmn + spn + eph + dp
model1  = tm + wnb + dmn + spn + eph + dp + bdn
signal=[]
for psr in psrs:
    if psr.name in ['J0437-4715','J1939+2134']:
        signal.append( model1( psr ) )
    else:
        signal.append( model0( psr ) )

PTA = signal_base.PTA(signal)
PTA.set_default_params(Dict)

In [ ]:
dp = dpdm_delay.dpdm_block(model='U1B')

model0  = tm + wnb + dmn + spn + eph 
model1  = tm + wnb + dmn + spn + eph + bdn
signal=[]
for psr in psrs:
    if psr.name in ['J0437-4715','J1939+2134']:
        signal.append( model1( psr ) )
    else:
        signal.append( model0( psr ) )

PTA_eph = signal_base.PTA(signal)
PTA_eph.set_default_params(Dict)



In [ ]:
outDir="bayes/"+"_m"+str(nmodes)+"_EPH"

xs = {par.name:par.sample() for par in PTA_eph.params}
for parname in Dict.keys():
    if parname in xs.keys():
        xs[parname] = Dict[parname]

x0 = np.hstack([xs[key] for key in sorted(xs.keys())])    
ndim = len(x0)
#============================

x0[0:11] = 0.

groups=[range(ndim)]

print(PTA_eph.get_lnlikelihood(x0))


covdiag = np.ones(ndim)*0.01

sampler = PTSampler(ndim, PTA_eph.get_lnlikelihood, PTA_eph.get_lnprior,
                cov=np.diag(covdiag),outDir=outDir,groups=groups,resume=False)

sampler.sample(x0, 100000, isave=1000)

## Additional part, MLH contribution by pulsar

In [ ]:
# original
dp = dpdm_delay.dpdm_block(model='U1B')


model0  = tm + wnb + dmn + spn + eph + dp
model1  = tm + wnb + dmn + spn + eph + dp + bdn
signal=[]
for psr in psrs:
    if psr.name in ['J0437-4715','J1939+2134']:
        signal.append( model1( psr ) )
    else:
        signal.append( model0( psr ) )

PTA = signal_base.PTA(signal)
PTA.set_default_params(Dict)

In [ ]:
model0  = tm + wnb + dmn + spn + dp + eph
model1  = tm + wnb + dmn + spn + dp + eph + bdn
ptas=[]
for psr in psrs:
    if psr.name in ['J0437-4715','J1939+2134']:
        pta = signal_base.PTA(model1(psr))
        pta.set_default_params(Dict)
        ptas.append(pta)
    else:
        pta = signal_base.PTA(model0(psr))
        pta.set_default_params(Dict)
        ptas.append(pta)
        
dp = dpdm_delay.dpdm_block(model='U1B')


In [ ]:
# load mlh results

u1b_n106 = np.load('bayes/u1b_n106.npy')[0][0]
u1b_n106 = PTA.map_params(u1b_n106)

u1b_n091 = np.load('bayes/u1b_n091.npy')[0][0]
u1b_n091 = PTA.map_params(u1b_n091)

u1b_n105 = np.load('bayes/u1b_n105.npy')[0][0]
u1b_n105 = PTA.map_params(u1b_n105)

u1b_n102 = np.load('bayes/u1b_n102.npy')[0][0]
u1b_n102 = PTA.map_params(u1b_n102)

Null = {par.name:0 for par in PTA.params}
Null['x_dp_log10_eps']=-80

In [ ]:
print(PTA.get_lnlikelihood(u1b_n091),PTA.get_lnlikelihood(Null))

mlhs1=[]
for pta in ptas:
    mlhs1.append(pta.get_lnlikelihood(u1b_n106)-pta.get_lnlikelihood(Null) )
print(mlhs1)


mlhs2=[]
for pta in ptas:
    mlhs2.append(pta.get_lnlikelihood(u1b_n091)-pta.get_lnlikelihood(Null) )
print(mlhs2)

mlhs3=[]
for pta in ptas:
    mlhs3.append(pta.get_lnlikelihood(u1b_n105)-pta.get_lnlikelihood(Null) )
print(mlhs3)

mlhs4=[]
for pta in ptas:
    mlhs4.append(pta.get_lnlikelihood(u1b_n102)-pta.get_lnlikelihood(Null) )
print(mlhs4)

names=[psr.name for psr in psrs]
print(names)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(mlhs1,'.')
plt.plot(mlhs2,'.')
plt.plot(mlhs3,'.')
plt.plot(mlhs4,'.')

In [ ]:
print(names[0],names[4],names[20],names[21],names[24],names[25])

In [ ]:
mlhs1=[]
for pta in ptas:
    mlhs1.append(pta.get_lnlikelihood(u1b_n106)-pta.get_lnlikelihood(Null) )
print(mlhs1)

In [2]:
!jupyter nbconvert --to python bayes_limit.ipynb

[NbConvertApp] Converting notebook bayes_limit.ipynb to python
[NbConvertApp] Writing 8217 bytes to bayes_limit.py
